# Complete Guide to Pydantic: From Beginner to Advanced

## What is Pydantic?
Pydantic is a Python library that provides **data validation** and **parsing** using Python type hints. It's the foundation for FastAPI and LangChain.

### Key Benefits:
- ✅ **Type Safety**: Automatic validation based on type hints
- ✅ **Data Parsing**: Convert JSON/dict to Python objects
- ✅ **Serialization**: Convert Python objects back to JSON/dict
- ✅ **Error Handling**: Clear validation error messages
- ✅ **Performance**: Built on Rust for speed (v2+)

### Learning Path:
1. Basic Models & Validation
2. Advanced Field Types
3. OOP Integration (Inheritance, Polymorphism, Abstraction)
4. Validators & Custom Logic
5. Serialization & Deserialization
6. Performance & Memory Management
7. Integration Patterns for FastAPI/LangChain

In [ ]:
!pip install pydantic
!pip install pydantic[email]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 8.1 MB/s eta 0:00:00


## 🧩 What is Pydantic?

Pydantic is a data validation and parsing library based on Python type annotations. It lets you:

- Create models (schemas) using standard Python classes
- Automatically validate and convert types
- Serialize and parse data easily
- Raise meaningful errors if data is invalid

> "Think of Pydantic as a smart gatekeeper for data coming in and out of your system."

In [ ]:
from pydantic import BaseModel

class User(BaseModel):
    id: int
    name: str
    email: str

# Auto-validation
data = {'id': '123', 'name': 'dhruv', 'email': 'dhruv@example.com'}
user = User(**data)
print(user)

id=123 name='dhruv' email='dhruv@example.com'


## 🔐 Encapsulation with Pydantic
Encapsulation hides internal implementation. Pydantic helps expose only validated, safe fields.


In [ ]:
from pydantic import BaseModel, PrivateAttr

class Account(BaseModel):
    username: str
    # Use PrivateAttr to define a private attribute
    _password: str = PrivateAttr()

    def check_password(self, input_pass: str) -> bool:
        # Access the private attribute directly
        return self._password == input_pass

# When creating the instance, assign the value to the private attribute after creation
acc = Account(username="dhruv")
acc._password = "secret123" # Assign the value to the private attribute

print(acc.username)
print("Password Valid:", acc.check_password("secret123"))

dhruv
Password Valid: True


In [ ]:
from pydantic import BaseModel, EmailStr, PrivateAttr

# Define a Pydantic model with encapsulation
class User(BaseModel):
    username: str
    email: EmailStr
    _secret_token: str = PrivateAttr(default='hidden_token')  # Encapsulated private attribute

    def get_token(self) -> str:
        """Access private token (encapsulation)."""
        return self._secret_token

# Example usage
try:
    user = User(username="dhruv", email="dhruv@example.com")
    print(f"User: {user.username}, Email: {user.email}")
    print(f"Secret Token: {user.get_token()}")  # Accessing private attribute via method
    # Invalid email example
    invalid_user = User(username="dhruv", email="invalid_email")
except ValueError as e:
    print(f"Validation Error: {e}")

User: dhruv, Email: dhruv@example.com
Secret Token: hidden_token
Validation Error: 1 validation error for User
email
  value is not a valid email address: An email address must have an @-sign. [type=value_error, input_value='invalid_email', input_type=str]


## 🧬 Inheritance with Pydantic
You can extend Pydantic models using classic inheritance.

In [ ]:
class BaseEmployee(BaseModel):
    id: int
    name: str

class Manager(BaseEmployee):
    department: str

mgr = Manager(id=1, name="dhruv", department="AI")
print(mgr)

id=1 name='dhruv' department='AI'


## 🧰 Abstract Classes + Pydantic Models
Mix Pydantic with abstract classes using `abc`.

In [ ]:
from abc import ABC, abstractmethod

class AbstractEntity(ABC):
    @abstractmethod
    def summary(self): pass

class Book(BaseModel, AbstractEntity):
    title: str
    author: str

    def summary(self):
        return f"{self.title} by {self.author}"

b = Book(title="Deep Learning", author="Ian Goodfellow")
print(b.summary())

Deep Learning by Ian Goodfellow


## 🎭 Polymorphism with Overriding and Operator Overloading
Use dunder methods with Pydantic to add behavior to models.


In [ ]:
class Vector(BaseModel):
    x: int
    y: int

    def __add__(self, other):
        return Vector(x=self.x + other.x, y=self.y + other.y)

v1 = Vector(x=1, y=2)
v2 = Vector(x=3, y=4)
print(v1 + v2)

x=4 y=6


In [ ]:
from pydantic import BaseModel

# Base model
class Vehicle(BaseModel):
    brand: str
    speed: float

    def description(self) -> str:
        """Base description method."""
        return f"{self.brand} vehicle with speed {self.speed} km/h"

# Subclass with polymorphism
class Car(Vehicle):
    seats: int

    def description(self) -> str:
        """Override description for Car (polymorphism)."""
        base_desc = super().description()
        return f"{base_desc}, {self.seats} seats"

# Subclass with polymorphism
class Truck(Vehicle):
    cargo_capacity: float

    def description(self) -> str:
        """Override description for Truck (polymorphism)."""
        base_desc = super().description()
        return f"{base_desc}, {self.cargo_capacity} tons capacity"

# Example usage
car = Car(brand="Toyota", speed=180.0, seats=5)
truck = Truck(brand="Volvo", speed=120.0, cargo_capacity=10.0)
print(car.description())
print(truck.description())

Toyota vehicle with speed 180.0 km/h, 5 seats
Volvo vehicle with speed 120.0 km/h, 10.0 tons capacity


## Abstraction with Pydantic and Iterables

**Concept**: Abstraction (OOP concept) hides implementation details, exposing only essential interfaces. Pydantic models can use abstract base classes (ABCs) and support iterables for processing collections.

**Use Case**: Managing a collection of items in an e-commerce API, with abstract methods for processing.

**Theory**:
- Use `abc.ABC` for abstract base classes.
- Implement `__iter__` to make models iterable.
- Use Pydantic for validating collection data.

**Example**: An abstract `Item` model with an iterable `Order` model.

In [ ]:
from pydantic import BaseModel
from abc import ABC, abstractmethod
from typing import List

# Abstract base class
class ItemBase(BaseModel, ABC):
    name: str
    price: float

    @abstractmethod
    def get_details(self) -> str:
        """Abstract method for item details."""
        pass

# Concrete class
class Item(ItemBase):
    def get_details(self) -> str:
        return f"{self.name}: ${self.price}"

# Iterable order model
class Order(BaseModel):
    items: List[Item]

    def __iter__(self):
        """Make Order iterable over items."""
        return iter(self.items)

# Example usage
order = Order(items=[
    Item(name="Laptop", price=999.99),
    Item(name="Mouse", price=29.99)
])
for item in order:
    print(item.get_details())

Laptop: $999.99
Mouse: $29.99


## 🧙 Decorators + Closures with Pydantic Models
Use decorators to log or validate behavior around models.


In [ ]:
def log_model_creation(func):
    def wrapper(*args, **kwargs):
        print("Model is being created...")
        return func(*args, **kwargs)
    return wrapper

@log_model_creation
def create_user(name):
    class User(BaseModel):
        name: str
    return User(name=name)

print(create_user("Dhruv"))

Model is being created...
name='Dhruv'


## 🔄 Iterators and Generators in Pydantic Use Cases
Use generators to stream model creation.

In [ ]:
class DataItem(BaseModel):
    id: int
    value: float

def data_generator(n):
    for i in range(n):
        yield DataItem(id=i, value=i * 1.1)

for item in data_generator(3):
    print(item)

id=0 value=0.0
id=1 value=1.1
id=2 value=2.2


## Custom Validators with Decorators

**Concept**: Pydantic allows custom validation logic using the `@field_validator` decorator. Decorators are a powerful Python feature for modifying function behavior.

**Use Case**: Validating complex data (e.g., password strength) in an API or user registration system.

**Closures** allow encapsulating logic
- Use closures to create reusable validation logic.

**Theory**:
- Use `@field_validator` to define custom validation for specific fields.
- Combine with regular expressions (RegEx) for pattern matching.
- Decorators wrap validation logic, keeping code modular.

**Example**: A `UserRegistration` model with a password validator using RegEx, showcasing decorators.

In [ ]:
from pydantic import BaseModel, field_validator
import re

# Define a Pydantic model with custom validator
class UserRegistration(BaseModel):
    username: str
    password: str

    @field_validator('password')
    def validate_password(cls, value):
        """Ensure password is at least 8 characters with a number and special character."""
        pattern = r'^(?=.*[0-9])(?=.*[!@#$%^&*]).{8,}$'
        if not re.match(pattern, value):
            raise ValueError('Password must be 8+ characters with a number and special character')
        return value

# Example usage
try:
    user = UserRegistration(username="alice", password="Secure#123")
    print(f"Valid User: {user}")
    # Invalid password example
    invalid_user = UserRegistration(username="bob", password="weak")
except ValueError as e:
    print(f"Validation Error: {e}")

Valid User: username='alice' password='Secure#123'
Validation Error: 1 validation error for UserRegistration
password
  Value error, Password must be 8+ characters with a number and special character [type=value_error, input_value='weak', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error


## 🪄 Pydantic with Lambda, Map, Filter
Functional programming helps transform lists of models.


In [ ]:
users = [
    {"id": 1, "name": "Dhruv"},
    {"id": 2, "name": "Arya"},
    {"id": 3, "name": "Sid"},
]

class SimpleUser(BaseModel):
    id: int
    name: str

user_objs = list(map(SimpleUser.parse_obj, users))
filtered = list(filter(lambda u: u.id % 2 != 0, user_objs))
print(filtered)

[SimpleUser(id=1, name='Dhruv'), SimpleUser(id=3, name='Sid')]


<ipython-input-16-45a34dfc0347>:11: PydanticDeprecatedSince20: The `parse_obj` method is deprecated; use `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  user_objs = list(map(SimpleUser.parse_obj, users))


In [ ]:
from pydantic import BaseModel, field_validator
from typing import Callable

# Closure for dynamic validation
def create_code_validator(prefix: str) -> Callable[[str], str]:
    def validate_code(value: str) -> str:
        if not value.startswith(prefix):
            raise ValueError(f"Code must start with {prefix}")
        return value
    return validate_code

# Pydantic model with closure and lambda
class Discount(BaseModel):
    code: str
    amount: float

    @field_validator('code')
    def validate_discount_code(cls, value):
        """Use closure and lambda for validation."""
        validator = create_code_validator("DISC")
        # Lambda to check length
        if not (lambda x: len(x) >= 6)(value):
            raise ValueError("Code must be 6+ characters")
        return validator(value)

# Example usage
try:
    discount = Discount(code="DISC2023", amount=10.0)
    print(f"Valid Discount: {discount}")
    invalid_discount = Discount(code="SALE123", amount=5.0)
except ValueError as e:
    print(f"Validation Error: {e}")

Valid Discount: code='DISC2023' amount=10.0
Validation Error: 1 validation error for Discount
code
  Value error, Code must start with DISC [type=value_error, input_value='SALE123', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error


## 🔍 RegEx Field Validation
Use `constr` to define regex patterns.

In [ ]:
from typing import Annotated
from pydantic import BaseModel, StringConstraints

class EmailUser(BaseModel):
    # Use Annotated and StringConstraints for regex validation in Pydantic v2
    email: Annotated[str, StringConstraints(pattern=r'^\S+@\S+\.\S+$')]

# This should now work
EmailUser(email="valid@example.com")

# This will still raise a validation error as expected
# EmailUser(email="invalid-email")

EmailUser(email='valid@@example.com')

## 🧠 Deep vs Shallow Copy
Use `.copy()` from Pydantic to avoid mutations.

In [ ]:
class Product(BaseModel):
    id: int
    tags: list

original = Product(id=1, tags=["ai", "ml"])
copy = original.copy(deep=True)

copy.tags.append("cv")
print("Original:", original.tags)
print("Copy:", copy.tags)

Original: ['ai', 'ml']
Copy: ['ai', 'ml', 'cv']


<ipython-input-19-9dcbc4dfaa1b>:6: PydanticDeprecatedSince20: The `copy` method is deprecated; use `model_copy` instead. See the docstring of `BaseModel.copy` for details about how to handle `include` and `exclude`. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  copy = original.copy(deep=True)


## 📦 Mutable vs Immutable in Pydantic
You can freeze models to make them immutable.

In [ ]:
class ImmutableUser(BaseModel, frozen=True):
    name: str
    id: int

# user = ImmutableUser(name="A", id=1)
# user.id = 2  # ❌ Raises error

##Memory Profiling with sys.getsizeof

In [ ]:
import sys
from pydantic import BaseModel

class HeavyModel(BaseModel):
    data: list

obj = HeavyModel(data=[x for x in range(1000)])
print("Memory (bytes):", sys.getsizeof(obj.data))

Memory (bytes): 8056


## Memory Management and Optimization

**Concept**: Pydantic models can be optimized for memory usage, especially for large datasets. Python’s memory management (reference counting, garbage collection) impacts performance.

**Use Case**: Processing large datasets in an API, optimizing memory with shallow vs. deep copies.

**Theory**:
- Use `model_validate_json` for faster JSON parsing.
- Understand shallow vs. deep copies to manage mutable objects.
- Python’s reference counting increments/decrements object references; garbage collection handles cyclic references.

**Example**: Optimizing a Pydantic model for large data with deep copy.

In [ ]:
from pydantic import BaseModel
from copy import deepcopy
import sys

# Pydantic model for large dataset
class DataPoint(BaseModel):
    id: int
    values: list[float]

# Create a large dataset
data = {"id": 1, "values": [1.0, 2.0, 3.0] * 1000}

# Shallow copy (shares references to mutable objects)
shallow_copy = data.copy()
print(f"Shallow copy memory: {sys.getsizeof(shallow_copy)} bytes")

# Deep copy (independent objects)
deep_copy = deepcopy(data)
print(f"Deep copy memory: {sys.getsizeof(deep_copy)} bytes")

# Validate with Pydantic
data_point = DataPoint.model_validate_json(str(data).replace("'", '"'))
print(f"Validated DataPoint: {data_point.id}, First few values: {data_point.values[:5]}")

Shallow copy memory: 184 bytes
Deep copy memory: 184 bytes
Validated DataPoint: 1, First few values: [1.0, 2.0, 3.0, 1.0, 2.0]


## 🧪 Pydantic + Pytest Unit Test
Write test cases for model behavior.

**Concept**: Pydantic models should be tested to ensure validation logic works. Pytest is a robust testing framework for Python.

**Use Case**: Testing a Pydantic model for an API to ensure data integrity.

**Theory**:
- Use Pytest to write unit tests for Pydantic models.
- Test valid and invalid cases to verify validation logic.
- Use `pytest.raises` to check for expected errors.

**Example**: Testing a `Product` model with Pytest.

In [ ]:
from pydantic import BaseModel, field_validator
import pytest

# Pydantic model
class Product(BaseModel):
    name: str
    price: float

    @field_validator('price')
    def price_positive(cls, value):
        if value <= 0:
            raise ValueError("Price must be positive")
        return value

# Pytest tests
def test_valid_product():
    product = Product(name="Book", price=29.99)
    assert product.name == "Book"
    assert product.price == 29.99

def test_invalid_price():
    with pytest.raises(ValueError):
        Product(name="Book", price=-10.0)

# Run tests (in Colab, use !pytest)
# Save this code to a file (e.g., test_product.py) and run: !pytest test_product.py
print("Save this code to test_product.py and run 'pytest test_product.py' locally.")

Save this code to test_product.py and run 'pytest test_product.py' locally.


In [ ]:
def test_user_model():
    from pydantic import BaseModel

    class U(BaseModel):
        id: int
        name: str

    u = U(id=1, name="Test")
    assert u.id == 1

test_user_model()

## Advanced Pydantic Demonstration
### ==============================
### This script demonstrates a comprehensive use of Pydantic with OOP concepts,
### advanced Python features, and integration patterns for FastAPI/LangChain.

In [ ]:
from pydantic import BaseModel, EmailStr, field_validator, ValidationError
from typing import List, Optional
from abc import ABC, abstractmethod
import time
import gc
import json
from functools import wraps
import pytest

# === Utility Classes ===
class PerformanceTimer:
    """Context manager for measuring execution time."""
    def __init__(self, operation_name):
        self.operation_name = operation_name
        self.start_time = None

    def __enter__(self):
        self.start_time = time.time()
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        elapsed = time.time() - self.start_time
        print(f"   ⏱️ {self.operation_name} took {elapsed:.4f}s")

# === Pydantic Models ===
class UserCreateRequest(BaseModel):
    """Pydantic model for user creation with validation (Encapsulation)."""
    username: str
    email: EmailStr
    password: str
    full_name: str
    age: int

    @field_validator('password')
    def validate_password(cls, value):
        """Custom validator for password strength using RegEx."""
        import re
        pattern = r'^(?=.*[0-9])(?=.*[!@#$%^&*]).{8,}$'
        if not re.match(pattern, value):
            raise ValueError("Password must be 8+ characters with a number and special character")
        return value

# === Inheritance and Polymorphism ===
class Animal(BaseModel, ABC):
    """Abstract base class for animals (Abstraction)."""
    name: str
    age: int
    weight: float

    @abstractmethod
    def make_sound(self) -> str:
        """Abstract method for animal sound (Polymorphism)."""
        pass

class Dog(Animal):
    """Dog model with specific attributes."""
    breed: str
    is_trained: bool

    def make_sound(self) -> str:
        return f"{self.name} says Woof!"

class Cat(Animal):
    """Cat model with specific attributes."""
    indoor_only: bool

    def make_sound(self) -> str:
        return f"{self.name} says Meow!"

class Bird(Animal):
    """Bird model with specific attributes."""
    wing_span: float

    def make_sound(self) -> str:
        return f"{self.name} says Chirp!"

# === Observer Pattern for Repository ===
class Observer(ABC):
    """Abstract observer for repository events."""
    @abstractmethod
    def update(self, event: str, data: dict):
        pass

class LoggingObserver(Observer):
    """Concrete observer that logs events."""
    def update(self, event: str, data: dict):
        print(f"   📝 Log: {event} - {data}")

class ObservableUserRepository:
    """Repository pattern with observer (Encapsulation)."""
    def __init__(self):
        self._observers = []
        self._users = []

    def add_observer(self, observer: Observer):
        self._observers.append(observer)

    def _notify_observers(self, event: str, data: dict):
        for observer in self._observers:
            observer.update(event, data)

    def create(self, username: str, email: str) -> UserCreateRequest:
        user = UserCreateRequest(
            username=username,
            email=email,
            password="Default123!",
            full_name="Default Name",
            age=25
        )
        self._users.append(user)
        self._notify_observers("user_created", {"username": username})
        return user

# === Optimized Model ===
class OptimizedModel(BaseModel):
    """Optimized Pydantic model for performance."""
    id: int
    is_active: bool
    score: float
    name: str

    class Config:
        """Optimize for performance."""
        validate_assignment = True
        use_enum_values = True
        json_encoders = {float: lambda v: round(v, 2)}

# === Processing Chain with Generators ===
class ProcessingChain:
    """Processing chain simulating LangChain-like workflow."""
    def __init__(self):
        self.steps = []
        self.total_processing_time = 0.0

    def add_step(self, step_name: str, input_data: dict):
        """Add a processing step."""
        self.steps.append({"name": step_name, "input": input_data, "status": "pending"})
        return len(self.steps) - 1

    def complete_step(self, step_id: int, output_data: dict, duration: float):
        """Complete a step and update processing time."""
        self.steps[step_id].update({"output": output_data, "status": "completed"})
        self.total_processing_time += duration

    def finalize(self):
        """Generator to yield completed steps."""
        for step in self.steps:
            if step["status"] == "completed":
                yield step

# === Main Demonstration Class ===
class AdvancedPydanticDemo:
    """
    Final demonstration combining all learned concepts:
    - Inheritance & Polymorphism
    - Validation & Custom Logic
    - Serialization & Performance
    - Integration Patterns
    - Testing & Optimization
    """

    def __init__(self):
        self.models_created = 0
        self.validation_errors = 0

    def demonstrate_all_concepts(self):
        """Demonstrate all Pydantic concepts in one cohesive example."""

        print("🎓 PYDANTIC MASTERY DEMONSTRATION")
        print("=" * 50)

        # 1. Basic Model Creation with Validation
        print("\n1️⃣ Basic Models & Validation:")
        try:
            user = UserCreateRequest(
                username="advanced_user",
                email="advanced@example.com",
                password="SuperSecure123!",
                full_name="Advanced Pydantic User",
                age=30
            )
            self.models_created += 1
            print(f"   ✅ Created user: {user.username}")
        except ValidationError as e:
            self.validation_errors += 1
            print(f"   ❌ Validation failed: {e}")

        # 2. Inheritance & Polymorphism
        print("\n2️⃣ Inheritance & Polymorphism:")
        animals = [
            Dog(name="Rex", age=5, weight=30.0, breed="German Shepherd", is_trained=True),
            Cat(name="Fluffy", age=3, weight=5.0, indoor_only=True),
            Bird(name="Tweety", age=2, weight=0.5, wing_span=0.4)
        ]

        for animal in animals:
            print(f"   🔊 {animal.make_sound()}")
            self.models_created += 1

        # 3. Generic Patterns & Design
        print("\n3️⃣ Generic Patterns:")
        user_repo = ObservableUserRepository()
        user_repo.add_observer(LoggingObserver())

        created_user = user_repo.create(
            username="pattern_user",
            email="patterns@example.com"
        )
        print(f"   📊 Repository created user: {created_user.username}")
        self.models_created += 1

        # 4. Performance & Memory Management
        print("\n4️⃣ Performance Optimization:")
        with PerformanceTimer("Model operations"):
            optimized_models = [
                OptimizedModel(
                    id=i,
                    is_active=True,
                    score=95.5,
                    name=f"Optimized_{i}"
                )
                for i in range(100)  # Create 100 models
            ]
            self.models_created += 100

        # 5. Serialization
        print("\n5️⃣ Serialization:")
        sample_model = optimized_models[0]
        json_data = sample_model.json()
        restored_model = OptimizedModel.parse_raw(json_data)
        print(f"   🔄 Serialized and restored: {restored_model.name}")

        # 6. Integration Patterns
        print("\n6️⃣ Integration Patterns:")
        chain = ProcessingChain()
        step = chain.add_step("demo_step", {"input": "demonstration"})
        chain.complete_step(step, {"output": "success"}, 0.001)
        chain.finalize()
        print(f"   ⛓️ Processing chain completed in {chain.total_processing_time:.4f}s")

        # 7. Memory Management
        print("\n7️⃣ Memory Management:")
        initial_objects = len(gc.get_objects())

        # Create and delete objects
        temp_models = [OptimizedModel(id=i, is_active=False, score=80.0, name=f"Temp_{i}") for i in range(50)]
        self.models_created += 50
        del temp_models  # Explicitly delete to free memory
        gc.collect()  # Force garbage collection
        final_objects = len(gc.get_objects())
        print(f"   🗑️ Objects before: {initial_objects}, after GC: {final_objects}")

        # 8. Deep Copy vs Shallow Copy
        print("\n8️⃣ Deep Copy vs Shallow Copy:")
        import copy
        original = OptimizedModel(id=999, is_active=True, score=99.9, name="Original")
        shallow_copy = copy.copy(original)  # Shares nested objects
        deep_copy = copy.deepcopy(original)  # Fully independent
        shallow_copy.name = "ShallowCopy"
        deep_copy.name = "DeepCopy"
        print(f"   Original name: {original.name}")  # Unchanged
        print(f"   Shallow copy name: {shallow_copy.name}")
        print(f"   Deep copy name: {deep_copy.name}")

        # 9. Summary
        print("\n9️⃣ Summary:")
        print(f"   Total models created: {self.models_created}")
        print(f"   Total validation errors: {self.validation_errors}")

# === Testing with Pytest ===
def test_advanced_demo():
    """Test the AdvancedPydanticDemo class."""
    demo = AdvancedPydanticDemo()
    demo.demonstrate_all_concepts()
    assert demo.models_created >= 152, "Should create at least 152 models"
    assert demo.validation_errors == 0, "No validation errors expected"

def test_invalid_user():
    """Test validation error handling."""
    with pytest.raises(ValidationError):
        UserCreateRequest(
            username="invalid",
            email="not_an_email",
            password="weak",
            full_name="Invalid User",
            age=-1
        )

# === Run the Demo ===
if __name__ == "__main__":
    demo = AdvancedPydanticDemo()
    demo.demonstrate_all_concepts()
    # To run tests, save this file and run: pytest advanced_pydantic_demo.py
    print("\nRun 'pytest advanced_pydantic_demo.py' to execute tests.")

🎓 PYDANTIC MASTERY DEMONSTRATION

1️⃣ Basic Models & Validation:
   ✅ Created user: advanced_user

2️⃣ Inheritance & Polymorphism:
   🔊 Rex says Woof!
   🔊 Fluffy says Meow!
   🔊 Tweety says Chirp!

3️⃣ Generic Patterns:
   📝 Log: user_created - {'username': 'pattern_user'}
   📊 Repository created user: pattern_user

4️⃣ Performance Optimization:
   ⏱️ Model operations took 0.0002s

5️⃣ Serialization:
   🔄 Serialized and restored: Optimized_0

6️⃣ Integration Patterns:
   ⛓️ Processing chain completed in 0.0010s

7️⃣ Memory Management:
   🗑️ Objects before: 142737, after GC: 142543

8️⃣ Deep Copy vs Shallow Copy:
   Original name: Original
   Shallow copy name: ShallowCopy
   Deep copy name: DeepCopy

9️⃣ Summary:
   Total models created: 155
   Total validation errors: 0

Run 'pytest advanced_pydantic_demo.py' to execute tests.


<ipython-input-37-5bef926fc564>:229: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  json_data = sample_model.json()
<ipython-input-37-5bef926fc564>:230: PydanticDeprecatedSince20: The `parse_raw` method is deprecated; if your data is JSON use `model_validate_json`, otherwise load the data then use `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  restored_model = OptimizedModel.parse_raw(json_data)


In [ ]:
#Act as a Python Expert who have worked on various advance concepts of Pydantic Libraries in Python with various Real life like use cases. now I am learning frameworks like fastapi and langchain and in that I have seen Pydantic lybraries been used in various use case time to time, so I though I should Learn Pydantic for myself. Now as someone who never know anything about Pydantic. help me understand and learn best practices of Pydantic library used on regular bases, and also help me learn how its useful while using in different use case like in OOPs concepts dependent related projects or APIs or decorater heavy project etc., things I need to know in Pydantic before learning langchain and FastAPI. for teaching me all this you can help me create an google colab notebook ipynb file where you are first explaining a component, concept or practices, then explaining it through a simple program through sample code that can help me understand those component, concept or practices completely. make sure to create proper markdown and comments with flowchart(if required and can use tags and all for masrkdown) so I can understand theory, concepts, flow and maths that works behind during that component, concept or practices, make sure to keep the explaination brief short and beginner-friendly. Make sure to keep the examples you provide for the respective pydantic component, concept or practices diverse. use some example that utilises one of any OOPs concepts (Encapsulation, Abstraction {abstract method/class}, Inheritence {different types}, Polymorphism (overriding or constructor{__init__} or destructor{__del__} or OperatorOverloading)) in defferent examples but must cover all the mentioned OOPs concepts through various examples, utilise super/self in some example, utilise decorators, Closures in some examples, Iterables, Iterators, Generators in some example, Advance Python Concepts (Lambda Operator,Filter, Reduce, Map, Recursive Functions, RegEx, Memory Management, Garbage Collection, How does reference counting work in Python?, How dynamic typing works?, Mutable & Immutable objects, Memory Profiling, Deep Copy vs Shallow Copy, Optimization Tips for Python Code, How Python store the integers in the memory?, Testing with Pytest, DocTests, UnitTests) in some example, All I want to say is keep the examples diverse for different component seperate for all the components I wrote with pydantic, concept or practices of pydantic. Provide Cell by cell code and markdowns(appropriate to colab) I will copy paste it myself. Can Search on web for details.

##Pydantic with LangChain: Structured AI Outputs

**Concept**: Pydantic is used in LangChain to structure and validate outputs from large language models (LLMs), ensuring consistent data formats.

**Use Case**: Structuring AI-generated responses for a chatbot.

**Theory**:
- Define Pydantic models for expected LLM output formats.
- Use with LangChain’s `with_structured_output` to enforce structure.
- Requires Pydantic V2 for modern LangChain compatibility.

**Example**: A Pydantic model for structuring chatbot responses. (Note: LangChain requires an LLM API key; this is a simplified example.)

In [ ]:
!pip install langchain langchain-openai

from pydantic import BaseModel
from langchain_openai import ChatOpenAI

# Pydantic model for structured output
class ChatResponse(BaseModel):
    response: str
    sentiment: str

# Mock LangChain setup (replace with actual LLM API key)
try:
    model = ChatOpenAI()
    structured_model = model.with_structured_output(ChatResponse)
    response = structured_model.invoke("Hello, how can I assist you?")
    print(f"Structured Response: {response}")
except Exception as e:
    print(f"Error: {e}. Set up an OpenAI API key to run this example.")